In [24]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('datasets/test_level_a.csv')
df.rename(columns={'label': 'true_label'}, inplace=True)
sampled_df =df.sample(n=10)
sampled_df.reset_index(drop=True, inplace=True)
sampled_df

,id,tweet,true_label
0,58543,"#Conservatives We mus pray for liberals. They trooly kno knot watt they do. I was brot up socialist democrat, by my wonderful, but, ignorant parents. I served inda military. I learnd I can serve Christ, too. Eye reel -eyesed CONSERVATIVISM was more closely aligned w God.",OFF
1,72867,#TEAM4THEBEEZ PLEASE SHARE THIS PUP DID NOTHING EXCEPT HIS OWNER DIED AND HE WAS #DUMPED2DIE AT SHELTER HE IS IN GRIEF FOR HIS OWNER AND TERRIFIED IN NOISY SHELTER PLEASE #NOKILLSHELTERNATION IS WHAT WE NEED BUT LETS FIND THIS POOR PUP A #HEROES2ANIMALS URL,NOT
2,66377,#FollowTheWhiteRabbit #QAnon #GreatAwakening #SpyGate #WalkAway QAnon #QArmy #MAGA #Google #MAGA Poland #FISADeclassification #DearProfessorFord URL Dem US Senator Corey Booker molesting a 15 year old in the 80's surfaces URL,NOT
3,76036,@USER @USER I agree with you. Read the first paragraph and struggled to get past the condescension. She is a lawyer with an educated opinion. There are other lawyers with differing educated opinions.,NOT
4,71071,"#HurricaneFlorence ... not good, she is sporting a pinhole eye while strengthening... URL",NOT
5,68363,4chan is Hunting Antifa URL via @USER,NOT
6,51589,@USER I see your point but the school shootings are extremely controversial with bickering over gun control vs gun rights. Just that the Nike controversy is the current one,NOT
7,87139,@USER this girl is so talented and she is my best friend because you and Ethan brought us together. Please notice this talent. @USER @USER @USER please💜🧡💜🧡💜🧡💜🧡💜🧡💜🧡💜🧡💜🧡 URL,NOT
8,21524,"#Khaleesi kinda watching #HIAC ! Wtf does she think she is, a person? Pffft pathetic URL",OFF
9,36455,#MyUniqueHeritage I Get My Irish From My Dad Side Of the Family Since my Last Name is Irish Cause I’m Very Proud Of Being Irish☘️💚 URL,NOT


In [25]:
df = pd.read_csv('predictions.csv')
df.drop(columns=['bert_label'], inplace=True)
df2 = pd.read_csv('predictions_svm_stems_other.csv')
df['svm_stems_other_label'] = df2['trad_label']
data = []

decriptions=["both correct", "GPT correct, SVM wrong", "GPT wrong, SWM correct", "both wrong"]
for i in range(0,4):
    for label in ["NOT", "OFF"]:
        if(i==0): filtered_df = df[(df['true_label'] == label) & (df['gpt_label'] == label) & (df['svm_stems_other_label'] == label)]
        if(i==1): filtered_df = df[(df['true_label'] == label) & (df['gpt_label'] == label) & (df['svm_stems_other_label'] != label)]
        if(i==2): filtered_df = df[(df['true_label'] == label) & (df['gpt_label'] != label) & (df['svm_stems_other_label'] == label)]
        if(i==3): filtered_df = df[(df['true_label'] == label) & (df['gpt_label'] != label) & (df['svm_stems_other_label'] != label)]
        if len(filtered_df) > 0:
            random_row = filtered_df.sample(n=1).to_dict(orient='records')[0]
            random_row['description']=decriptions[i]
            data.append(random_row)

df2 = pd.DataFrame(data)
df2

,id,tweet,true_label,gpt_label,svm_stems_other_label,description
0,11284,.@USER takes the season opener on Saturday. Highlights: URL,NOT,NOT,NOT,both correct
1,42192,#ArunJaitleyStepDown He is most shameless #FM in history of India and audacity and shamelessness with which is lies in public is disgrace to post.,OFF,OFF,OFF,both correct
2,13993,#Immigration Public Charge Rule has been law for 109 years U.S social safety net was designed to support U.S Citizens Not to finance foreigners lives in he U.S #Trump #MAGA . URL via @USER,NOT,NOT,OFF,"GPT correct, SVM wrong"
3,98916,"#ANTIFA are bedfellows with The Democratic Socialists of America (DSA) who #JamesOKeefe has exposed as being inserted throughout government, attempting to bring down #America #Trump #MAGA🇺🇸 #DeepState URL",OFF,OFF,NOT,"GPT correct, SVM wrong"
4,68371,#Democrats and #Liberals accuse #BrettKavanaugh and have no proof!! Meanwhile they supported #TedKennedy in his run for @USER and called him the lion of the Senate and he killed a woman. #wednesdaywisdom #news #trump URL,NOT,OFF,NOT,"GPT wrong, SWM correct"
5,23542,"#Resist traitors caught on video breaking our laws to further their #Communist #Socialism ways and slow down the #Trump agenda. They are the DC Chapter of Democrat Socialists of America. Forget anTifa, that is a distraction from the real Resistance. #MAGA URL",OFF,NOT,OFF,"GPT wrong, SWM correct"
6,16323,An American Tail really is one of the most underrated animations ever ever ever. Fuck I cried in this scene,NOT,OFF,OFF,both wrong
7,72401,"@USER * Puerto Rico is destroyed; more than 3,000 die * Trump visits, and throws paper rolls at people * 11 months later, Puerto Rico is still waiting for help * Trump says everything is fine, and his visit was a great success 🤔",OFF,NOT,NOT,both wrong


In [37]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df["true_label"])

tokenizer = AutoTokenizer.from_pretrained("openai-gpt")
if tokenizer.pad_token is None: tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model=AutoModelForSequenceClassification.from_pretrained('gpt_checkpoint/checkpoint-5958/')
pipeline=pipeline("text-classification", model=model, tokenizer=tokenizer)

Using pad_token, but it is not set yet.


In [53]:
text=input("Please input text: ")
result=pipeline(text)
print("text: ", text, "  label:",result[0]['label'],", confidence score:",result[0]['score'])

text:  today is raining   label: NOT , confidence score: 0.8648585081100464
